In [2]:
#训练好的模型进行预测，直观判断
import cv2
import glob
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.contrib.slim import nets
def get_next_batch(i=0,batch_size=1, images_dir='./test_img2'):
    if not os.path.exists(images_dir):
        raise ValueError('`images_dir` is not exist.')
    images = []
    labels = []
    image_files = np.array(glob.glob(os.path.join(images_dir, '*.jpg')))
    batch_size = min(batch_size, len(image_files)-i)
    selected_indices= np.arange(i,i+batch_size,1)
    selected_images_path = image_files[selected_indices]
    for image_path in selected_images_path:
        image = cv2.imread(image_path)
        #print(image_path)
        try:
            image = cv2.resize(image, (224, 224))
        except:
            image=np.zeros((224,224,3))
        label = image_path.split('/')[-1].split('_')[0]
        #label=0
        images.append(image)
        labels.append(int(label))
    images = np.array(images)
    labels = np.array(labels)
    return images,labels

images_dir='./predict'
with tf.Session() as sess:
    tf.set_random_seed(1)
    #saver = tf.train.import_meta_graph('./output_base/model.ckpt_214.meta')
    #saver.restore(sess, './output_base/model.ckpt_214')
    saver = tf.train.import_meta_graph('./output_base/1_2_3/model.ckpt_347.meta')
    saver.restore(sess, './output_base/1_2_3/model.ckpt_347')
    inputs = tf.get_default_graph().get_tensor_by_name('inputs:0')
    is_training=tf.get_default_graph().get_tensor_by_name('is_training:0')
    prediction = tf.get_default_graph().get_tensor_by_name('Softmax:0')
    image_files = np.array(glob.glob(os.path.join(images_dir, '*.jpg')))
    keep_prob_new = tf.get_default_graph().get_tensor_by_name('keep_prob:0')
    batchsize=1
    i=0
    out=[]
    label=[]
    while True:
        if i>=len(image_files):
            break
        images,labels=get_next_batch(i,batchsize,images_dir)
        i=i+batchsize
        pred = sess.run(prediction, feed_dict={inputs: images,is_training: False,keep_prob_new: 1.0})
        for id in range(len(pred)):
            img_new=images[id][..., -1::-1]
            #plt.imshow(img_new)    # 因为opencv读取进来的是bgr顺序呢的，而imshow需要的是rgb顺序，因此需要先反过来
            #plt.show()
            #print(labels[id])
            #print(pred)
            if pred[id][0]<=0.2:
                #print('1')
                out.append(1)
            else:
                #print('0')
                out.append(0)
            label.append(labels[id])
    recall=0
    acc=0
    len_recall=0
    id=0
    pre=0
    pre_all=0
    print(label,out)
    for i in range(len(label)):
        if label[i]==out[i]:
            acc+=1
        if label[i]==1:
            len_recall+=1
            if out[i]==1:
                recall+=1
        if out[i]==1:
            pre_all+=1
            if label[i]==1:
                pre+=1
    print('ACC is {},recall is {},presion is {}'.format(acc/len(label),recall/len_recall,pre/pre_all)) 
    #print('ACC is {}'.format(acc/len(label))) 
    print(len(label),acc,len_recall,recall)
    '''ACC is 0.9570637119113573,recall is 0.871244635193133,presion is 0.9950980392156863
    './output_base/1_3/model.ckpt_246.meta' ACC is 0.9918367346938776,recall is 0.9785407725321889,presion is 0.9956331877729258
685 655 222 200'''

INFO:tensorflow:Restoring parameters from ./output_base/1_2_3/model.ckpt_347
[1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0

In [1]:
# 预测标注好的数据
import cv2
import glob
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.contrib.slim import nets

sku_label={}
file=open('./manual/review_data_label_1_2_3.txt','r',encoding='utf8')
for line in file:
    sku=line.split('\t')[0]
    label=line.split('\t')[1]
    sku_label[sku]=int(label)
file.close()

def get_next_batch(i=0,batch_size=1, images_dir='./test_img2'):
    if not os.path.exists(images_dir):
        raise ValueError('`images_dir` is not exist.')
    images = []
    labels = []
    image_files = np.array(glob.glob(os.path.join(images_dir, '*.jpg')))
    batch_size = min(batch_size, len(image_files)-i)
    selected_indices= np.arange(i,i+batch_size,1)
    selected_images_path = image_files[selected_indices]
    for image_path in selected_images_path:
        image = cv2.imread(image_path)
        #print(image_path)
        try:
            sku = image_path.split('/')[-1].split('.')[0]
            label = sku_label[str(sku)]
            image = cv2.resize(image, (224, 224))
        except:
            #image=np.zeros((224,224,3))
            continue
        #label=0
        images.append(image)
        labels.append(int(label))
    images = np.array(images)
    labels = np.array(labels)
    return images,labels

images_dir='./manual/review_img'
with tf.Session() as sess:
    tf.set_random_seed(1)
    #saver = tf.train.import_meta_graph('./output_base/2/model.ckpt_260.meta')
    saver = tf.train.import_meta_graph('./output_base/1_3/model.ckpt_246.meta')
    #saver.restore(sess, './output_base/2/model.ckpt_260')
    saver.restore(sess, './output_base/1_3/model.ckpt_246')
    inputs = tf.get_default_graph().get_tensor_by_name('inputs:0')
    is_training=tf.get_default_graph().get_tensor_by_name('is_training:0')
    prediction = tf.get_default_graph().get_tensor_by_name('Softmax:0')
    image_files = np.array(glob.glob(os.path.join(images_dir, '*.jpg')))
    keep_prob_new = tf.get_default_graph().get_tensor_by_name('keep_prob:0')
    batchsize=64
    i=0
    out=[]
    label=[]
    while True:
        if i>=len(image_files):
            break  
        images,labels=get_next_batch(i,batchsize,images_dir)
        i=i+batchsize
        pred = sess.run(prediction, feed_dict={inputs: images,is_training: False,keep_prob_new: 1.0})
        for id in range(len(pred)):
            img_new=images[id][..., -1::-1]
            #plt.imshow(img_new)    # 因为opencv读取进来的是bgr顺序呢的，而imshow需要的是rgb顺序，因此需要先反过来
            #plt.show()
            #print(labels[id])
            #print(pred)
            if pred[id][0]<=0.25:
                #print('1')
                out.append(1)
            else:
                #print('0')
                out.append(0)
            label.append(labels[id])
    recall=0
    acc=0
    len_recall=0
    id=0
    pre=0
    pre_all=0
    #print(label,out)
    for i in range(len(label)):
        if label[i]==out[i]:
            acc+=1
        if label[i]==1:
            len_recall+=1                          
            if out[i]==1:
                recall+=1
        if out[i]==1:
            pre_all+=1
            if label[i]==1:
                pre+=1
    print('ACC is {},recall is {},presion is {}'.format(acc/len(label),recall/len_recall,pre/pre_all)) 
    #print('ACC is {}'.format(acc/len(label))) 
    print(len(label),acc,len_recall,recall)
''' base ACC is 0.822268772155978,recall is 0.8506024096385543,presion is 0.853573347662547
    ./output_base/model.ckpt_214.meta 用num1训练后的 ACC is 0.8196906219787302,recall is 0.7681392235609104,presion is 0.9189622037155669
    './output_base/1_3/model.ckpt_246.meta' ACC is 0.9286915819498281,recall is 0.8749378006302869,presion is 0.8151753979292227
    './output_base/1_2_3/model.ckpt_347.meta' ACC is 0.9286915819498281,recall is 0.9011444684027202,presion is 0.8005009577132753'''

INFO:tensorflow:Restoring parameters from ./output_base/1_3/model.ckpt_246
ACC is 0.9286915819498281,recall is 0.8749378006302869,presion is 0.8151753979292227
27346 25396 6029 5275


" base ACC is 0.822268772155978,recall is 0.8506024096385543,presion is 0.853573347662547\n    ./output_base/model.ckpt_214.meta 用num1训练后的 ACC is 0.8196906219787302,recall is 0.7681392235609104,presion is 0.9189622037155669\n    './output_base/1_3/model.ckpt_246.meta' ACC is 0.9286915819498281,recall is 0.8749378006302869,presion is 0.8151753979292227\n    './output_base/1_2_3/model.ckpt_347.meta' ACC is 0.9286915819498281,recall is 0.9011444684027202,presion is 0.8005009577132753"

In [ ]:
import pylab as plt
import cv2
import numpy as np
from keras.models import load_model

np.random.seed(1113)  # for reproducibility
import cv2
import os

def double(selected_images_path,path): 
    images=[]
    labels=[]
    for image_path in selected_images_path:
        try:
            image = cv2.imread(path+image_path)
            #label = int(image_path.split('_')[0])
            label=0
            if image is not None:
                images.append(image)
                labels.append(int(label))
        except:
            continue
    images = np.array(images)
    labels = np.array(labels)
    return images,labels

name='./test_img/'
#多张预测
imagelist = os.listdir(name)
x_test,y_test=double(imagelist,name)

print(y_test)
x_test_compare=x_test
x_test=x_test.reshape(-1,3,130,130)/255.
model = load_model('../CNN_model4')
recall=0
acc=0
len_recall=0
out=[]
y_pre=model.predict(x_test)
print(y_pre)
id=0
for item in y_pre:
    if y_test[id]==1:
        len_recall+=1
        if item[0]>0.5:
            recall+=1
    if item[0]>0.5:
        out.append(1)
        if y_test[id]==1:
            acc+=1
    else:
        out.append(0)
        if y_test[id]==0:
            acc+=1
    id+=1
#print('ACC is {},Recall is {}'.format(acc/len(y_test),recall/len_recall))
print('ACC is {}'.format(acc/len(y_test)))

for i in range(len(y_test)):
    I=x_test_compare[i]
    plt.imshow(I[..., -1::-1])    # 因为opencv读取进来的是bgr顺序呢的，而imshow需要的是rgb顺序，因此需要先反过来
    plt.show()
    print(out[i],'('+str(y_test[i])+')')

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
